<a href="https://colab.research.google.com/github/SushmaChandrachar/Gesture-recognition/blob/main/Final_submit_Sushma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
#from scipy.misc import imread, imresize
#from scipy.misc.pilutil import imread
from skimage.io import imread
from skimage.transform import resize
import datetime
import os
import pandas as pd
import tensorflow as tf

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [4]:
train_doc = np.random.permutation(open('/content/drive/My Drive/app/gesture_recognition/Project_data/train.csv').readlines())

#train_doc = np.random.permutation(open('C:/Users/susc/Documents/DL/gesture_recognition/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/content/drive/My Drive/app/gesture_recognition/Project_data/val.csv').readlines())
batch_size = 40

In [5]:
 img_idx = [*range(0,30,2)] #create a list of image numbers you want to use for a particular video
 x = len(img_idx)
 y = 100
 z = 100
 print(img_idx,x, y,z)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28] 15 100 100


## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [22]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [range(0,30,2)] #create a list of image numbers you want to use for a particular video
    x = len(img_idx)
    y = 100
    z = 100
    print(img_idx,x, y,z)

    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size
        for batch in range(num_batches):
          # we iterate over the number of batches
          batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
          batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
          for folder in range(batch_size):
            # iterate over the batch_size
            imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
            for idx,item in enumerate(img_idx):
              #  Iterate iver the frames/images of a folder to read them in
              image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
              image_resized= resize(image,(y,z),anti_aliasing=True)
              #crop the images and resize them. Note that the images are of 2 different shape 
              #and the conv3D will throw error if the inputs in a batch have different shapes
                    
              batch_data[folder,idx,:,:,0] = (image_resized[:,:,0])/255
              batch_data[folder,idx,:,:,1] = (image_resized[:,:,1])/255
              batch_data[folder,idx,:,:,2] = (image_resized[:,:,2])/255
                    
              batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        remaining = len(t)%batch_size
        if remaining > 0:
          batch_data = np.zeros((remaining,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
          batch_labels = np.zeros((remaining,5)) # batch_labels is the one hot representation of the output
          for folder in range(remaining):
            # iterate over the batch_size
            imgs = os.listdir(source_path+'/'+ t[folder + (batch*remaining)].split(';')[0]) # read all the images in the folder
            for idx,item in enumerate(img_idx):
               #  Iterate iver the frames/images of a folder to read them in
              image = imread(source_path+'/'+ t[folder + (batch*remaining)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
              image_resized= resize(image,(y,z),anti_aliasing=True)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
              batch_data[folder,idx,:,:,0] = (image_resized[:,:,0])/255
              batch_data[folder,idx,:,:,1] = (image_resized[:,:,1])/255
              batch_data[folder,idx,:,:,2] = (image_resized[:,:,2])/255
                    
              batch_labels[folder, int(t[folder + (batch*remaining)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do



In [23]:
train_path = '/content/drive/My Drive/app/gesture_recognition/Project_data/train'
train_gen = generator(train_path, train_doc, batch_size)

img,lab =next(train_gen)
import matplotlib.pyplot as plt
plt.figure(figsize=(15,10))

for x in range(0,15):
    plt.subplot(3,5,x+1)
    img2=img[0][x][:,:,:]
    plt.imshow(img2)
    plt.axis("off")

Source path =  /content/drive/My Drive/app/gesture_recognition/Project_data/train ; batch size = 40
[range(0, 30, 2)] 1 100 100


TypeError: ignored

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [8]:
curr_dt_time = datetime.datetime.now()
train_path = '/content/drive/My Drive/app/gesture_recognition/Project_data/train'
val_path = '/content/drive/My Drive/app/gesture_recognition/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [9]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [10]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [11]:
#Hyper Parameters

drop_val=0.25
input_sh=(x,y,z,3)

In [12]:
from tensorflow.keras.callbacks import EarlyStopping
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint1 = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
checkpoint2 = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.0001)
early_stopping = EarlyStopping(monitor='val_loss',patience=5,mode='auto',min_delta=0.001)
callbacks_list = [checkpoint1,checkpoint2, LR, early_stopping]
callbacks_list_noES = [checkpoint1,checkpoint2, LR]

In [13]:
#Number of steps per epoch for training and validation

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [14]:
#write your model here
sample_shape = (x,y,z,3)
no_classes = 5

# Create the model
conv3d_1 = Sequential()

conv3d_1.add(Conv3D(16, kernel_size=(3, 3, 3),input_shape=sample_shape,padding = 'same'))
conv3d_1.add(Activation('relu'))
conv3d_1.add(MaxPooling3D(pool_size=(2, 2, 2)))

conv3d_1.add(Conv3D(32, kernel_size=(3, 3, 3),padding = 'same'))
conv3d_1.add(Activation('relu'))
conv3d_1.add(MaxPooling3D(pool_size=(2, 2, 2)))

conv3d_1.add(Conv3D(64, kernel_size=(3, 3, 3),padding = 'same'))
conv3d_1.add(Activation('relu'))
conv3d_1.add(MaxPooling3D(pool_size=(2, 2, 2)))


conv3d_1.add(Flatten())

conv3d_1.add(Dense(512, activation='relu'))

conv3d_1.add(Dense(no_classes, activation='softmax'))

In [15]:
optimiser = optimizers.Adam()
conv3d_1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (conv3d_1.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 15, 100, 100, 16)  1312      
                                                                 
 activation (Activation)     (None, 15, 100, 100, 16)  0         
                                                                 
 max_pooling3d (MaxPooling3D  (None, 7, 50, 50, 16)    0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 7, 50, 50, 32)     13856     
                                                                 
 activation_1 (Activation)   (None, 7, 50, 50, 32)     0         
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 3, 25, 25, 32)    0         
 3D)                                                    

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [26]:
conv3d_1.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
             callbacks=callbacks_list, validation_data=val_generator,
             validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

StopIteration: ignored